In [2]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, SeparableConv2D, concatenate, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import AveragePooling2D, Input, Flatten, concatenate, Lambda, Input, Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model,Sequential
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import os

Using TensorFlow backend.


In [0]:
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = True
num_classes = 10

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [5]:
input_shape = x_train.shape[1:]
print(input_shape)

(32, 32, 3)


In [0]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255  

In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# def space_to_depth_x4(x):
#     return tf.space_to_depth(x, block_size=4)

In [0]:
def space_to_depth_x2(x):
  import tensorflow as tf
  return tf.space_to_depth(x, block_size=2)

In [10]:
inputs = Input(shape=(32, 32, 3,))

# Stage 1
x1 = SeparableConv2D(32,(5,5),border_mode='same', name='Layer1')(inputs)
x1 = BatchNormalization(name = 'bn_conv1')(x1)
x1 = Activation('relu')(x1)

# Stage 2
x2 = Conv2D(32, (5,5), strides=(1,1),border_mode='same', name='Layer_2', use_bias=False)(x1)
X2 = BatchNormalization(name = 'bn_conv2')(x2)
x2 = Activation('relu')(x2)

# Stage 3
x3 = Conv2D(32, (5,5), strides=(1,1), border_mode='same', name='Layer_3', use_bias=False)(x2)
X3 = BatchNormalization(name = 'bn_conv3')(x3)
x3 = Activation('relu')(x3)

#adding skip_connection of layer 3
skip_connection1 = x1
x3_concat =  concatenate([x3, x1])

# Stage 4
x4 = SeparableConv2D(32,(5,5), border_mode='same', name='Layer4')(x3_concat)
x4 = BatchNormalization(name = 'bn_conv4')(x4)
x4 = Activation('relu')(x4)

#adding skip_connection after layer 4
skip_connection2 = x1
x4_concat =  concatenate([x4, skip_connection2])

#max-pooling layer
mp1 = MaxPooling2D(pool_size=(2,2))(x4_concat)

# Stage 5
x5 = SeparableConv2D(32,(3,3), border_mode='same', name='Layer5')(mp1)
x5 = BatchNormalization(name = 'bn_conv5')(x5)
x5 = Activation('relu')(x5)
x5_concat = concatenate([x4,x1])
skip_con5= Lambda(space_to_depth_x2)(x5_concat)
x5_concat = concatenate([skip_con5, x5])

#Stage 6
x6 = Conv2D(32, (5,5), strides=(1,1), border_mode='same', name='Layer_6', use_bias=False)(x5_concat)
X6 = BatchNormalization(name = 'bn_conv6')(x6)
x6 = Activation('relu')(x6)

x6_temp = concatenate([x4,x3])
x6_temp = Lambda(space_to_depth_x2)(x6_temp)
x6_concat = concatenate([x6_temp, x5,x6])

#Stage 7
x7 = SeparableConv2D(32,(3,3), border_mode='same', name='Layer7')(x6_concat)
x7 = BatchNormalization(name = 'bn_conv7')(x7)
x7 = Activation('relu')(x7)
x7_temp = concatenate([x1, x3, x4])
x7_temp = Lambda(space_to_depth_x2)(x7_temp)
x7_concat = concatenate([x5, x6, x7_temp])

#Stage 8
x8 = SeparableConv2D(32,(5,5), border_mode='same', name='Layer8')(x7_concat)
x8 = BatchNormalization(name = 'bn_conv8')(x8)
x8 = Activation('relu')(x8)
x8_temp = concatenate([x1, x4])
x8_temp = Lambda(space_to_depth_x2)(x8_temp)
x8_concat = concatenate([x5, x7, x8_temp])

#Max-Pooling - 2
mp2 = MaxPooling2D(pool_size=(2,2))(x8_concat)
x6_temp = Lambda(space_to_depth_x2)(x6)
mp2_concat = concatenate([mp2, x6_temp])

# Stage 9
x9 = Conv2D(32, (5,5), strides=(1,1), border_mode='same', name='Layer_9', use_bias=False)(mp2_concat)
X9 = BatchNormalization(name = 'bn_conv9')(x9)
x9 = Activation('relu')(x9)
x9_temp = concatenate([x1, x2])
x9_temp = Lambda(space_to_depth_x2)(x9_temp)
x9_temp = concatenate([x9_temp, x7])
x9_temp = Lambda(space_to_depth_x2)(x9_temp)
x9_concat = concatenate([x9, x9_temp])

#Stage 10
x10 = SeparableConv2D(32,(5,5), border_mode='same', name='Layer10')(x9_concat)
x10 = BatchNormalization(name = 'bn_conv10')(x10)
x10 = Activation('relu')(x10)
x10_temp = concatenate([x2, x3])
x10_temp = Lambda(space_to_depth_x2)(x10_temp)
x10_temp = concatenate([x10_temp, x5])
x10_temp = Lambda(space_to_depth_x2)(x10_temp)
x10_temp = concatenate([x10_temp, x9])
x10_concat = concatenate([x10_temp, x10])

#Stage 11
x11 = Conv2D(32, (3,3), strides=(1,1), border_mode='same', name='Layer_11', use_bias=False)(x10_concat)
X11 = BatchNormalization(name = 'bn_conv9')(x11)
x11 = Activation('relu')(x11)
x11_temp = concatenate([x1, x3, x4])
x11_temp = Lambda(space_to_depth_x2)(x11_temp)
x11_temp = concatenate([x11_temp, x5])
x11_temp = Lambda(space_to_depth_x2)(x11_temp)
x11_temp = concatenate([x11_temp, x10])
x11_concat = concatenate([x11_temp, x11])

#stage 12
x12 = SeparableConv2D(32,(5,5), border_mode='same', name='Layer12')(x11_concat)
x12 = BatchNormalization(name = 'bn_conv12')(x12)
x12 = Activation('relu')(x12)
x12_temp = Lambda(space_to_depth_x2)(x4)
x12_temp = concatenate([x12_temp, x7])
x12_temp = Lambda(space_to_depth_x2)(x12_temp)
x12_temp = concatenate([x12_temp, x10])
x12_concat = concatenate([x12_temp, x12])
layer_last = Conv2D(10, (3,3), strides=(1,1), name='Layer_last', use_bias=False)(x12_concat)

flatten = GlobalAveragePooling2D()(layer_last)
#flatten = Flatten()(layer_last)
output = Activation('softmax')(flatten)

model = Sequential()
model = Model(inputs = [inputs], output = [output])

W0622 06:08:17.194133 140528821213056 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0622 06:08:17.230226 140528821213056 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32, (5, 5), name="Layer1", padding="same")`
  after removing the cwd from sys.path.
W0622 06:08:17.241065 140528821213056 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0622 06:08:17.298291 140528821213056 deprecation_w

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
Layer1 (SeparableConv2D)        (None, 32, 32, 32)   203         input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 32)   128         Layer1[0][0]                     
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
Layer_2 (C

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=100, verbose=1, validation_data=(x_test,y_test))
model.save('my_model.hdf5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 43s 853us/step - loss: 0.1146 - acc: 0.9590 - val_loss: 1.3235 - val_acc: 0.7293
Epoch 2/100
50000/50000 [==============================] - 41s 813us/step - loss: 0.1003 - acc: 0.9642 - val_loss: 1.7341 - val_acc: 0.6764
Epoch 3/100
50000/50000 [==============================] - 40s 809us/step - loss: 0.0895 - acc: 0.9688 - val_loss: 1.4110 - val_acc: 0.7159
Epoch 4/100
50000/50000 [==============================] - 41s 813us/step - loss: 0.0797 - acc: 0.9728 - val_loss: 1.6078 - val_acc: 0.6829
Epoch 5/100
50000/50000 [==============================] - 41s 813us/step - loss: 0.0843 - acc: 0.9693 - val_loss: 2.1807 - val_acc: 0.6280
Epoch 6/100
50000/50000 [==============================] - 41s 818us/step - loss: 0.0780 - acc: 0.9719 - val_loss: 1.6433 - val_acc: 0.6939
Epoch 7/100
50000/50000 [==============================] - 41s 813us/step - loss: 0.0707 - acc: 0.9754 - val_l

In [17]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 3s 305us/step
Test loss: 2.290043265914917
Test accuracy: 0.7137
